In [2]:
import pandas as pd
import datetime

# Load data

In [3]:
reestr_place = pd.read_csv('data/reestr_df.csv', encoding = 'cp1251',index_col=0)
# статистика по стране
reestr_place['Страна'] = reestr_place['Страна'].map(lambda x: 'Россия' if x == 'Россия' else 'Иностранцы')
reestr_place = reestr_place.merge(pd.get_dummies(reestr_place['Страна']), left_index=True, right_index=True).drop('Страна', 1)
# статистика по ЛС
reestr_place = reestr_place.merge(pd.get_dummies(reestr_place['Статус ЛС']), left_index=True, right_index=True).drop('Статус ЛС', 1)
# статистика по Масштаб
reestr_place = reestr_place.merge(pd.get_dummies(reestr_place['Масштаб']), left_index=True, right_index=True).drop('Масштаб', 1)
# статистика по Тип
reestr_place = reestr_place.merge(pd.get_dummies(reestr_place['Тип']), left_index=True, right_index=True).drop('Тип', 1)
# время
reestr_place['Дата документа'] = pd.DatetimeIndex(reestr_place['Информационное письмо'].map(lambda x: x.split('от ')[1]))
reestr_place['Год документа'] = pd.DatetimeIndex(reestr_place['Дата документа']).year
reestr_place['Месяц документа'] = pd.DatetimeIndex(reestr_place['Дата документа']).month
reestr_place['Месяц документа'] = reestr_place['Месяц документа'].astype('str') + '.' + reestr_place['Год документа'].astype('str')
reestr_place['Месяц документа'] = reestr_place['Месяц документа'].map(lambda x: datetime.datetime.strptime(x, '%m.%Y'))

In [5]:
# groupby stats
g_cols = reestr_place.drop(['Серия_y', 'Дата документа', 'Год документа', 'Месяц документа'], 1).loc[:, 'Иностранцы':].columns.values 
reestr_place= reestr_place.groupby(['place_id'], as_index = False)[g_cols].sum()
reestr_place['place_total_letters'] = reestr_place['Иностранцы'] + reestr_place['Россия']
for g in g_cols:
    reestr_place[g] = reestr_place[g] / reestr_place['place_total_letters']

# Save data

In [6]:
reestr_place.to_csv('data/fed_prepared.csv', encoding='cp1251', sep=';', index = False)